In [ ]:
# visualize_timeline_min.py
from pathlib import Path
import json
import matplotlib.pyplot as plt

BASE = Path.cwd()
JSON_DIR = BASE / "json"


def load_words(p: Path):
    d = json.loads(p.read_text(encoding="utf-8"))
    ws = []
    segs = d.get("segments")
    if isinstance(segs, list):
        for seg in segs:
            for w in seg.get("words", []):
                s, e = float(w.get("start", 0)), float(w.get("end", 0))
                if e > s:
                    ws.append({"start": s, "end": e})
    if not ws and isinstance(d.get("words"), list):
        for w in d["words"]:
            s, e = float(w.get("start", 0)), float(w.get("end", 0))
            if e > s:
                ws.append({"start": s, "end": e})
    ws.sort(key=lambda x: (x["start"], x["end"]))
    return ws


def find_gaps(ws):
    if not ws:
        return []
    gaps = []
    if ws[0]["start"] > 0:
        gaps.append((0.0, ws[0]["start"]))
    for i in range(1, len(ws)):
        a, b = ws[i - 1]["end"], ws[i]["start"]
        if b > a:
            gaps.append((a, b))
    return gaps


def plot_timeline(file_id="input"):
    js = JSON_DIR / f"{file_id}.json"
    ws = load_words(js)
    if not ws:
        print("No words")
        return
    T = max(ws[-1]["end"], ws[0]["start"] + 0.1)
    fig, ax = plt.subplots(figsize=(max(8, T / 5), 1.6))
    for w in ws:
        ax.barh(0, w["end"] - w["start"], left=w["start"], height=0.6, color="blue")
    for s, e in find_gaps(ws):
        ax.barh(0, e - s, left=s, height=0.6, color="red")
    ax.set_xlim(0, T)
    ax.set_yticks([])
    ax.set_xlabel("time (s)")
    ax.set_title(f"{file_id}: speech(blue) / gaps(red)")
    out = BASE / f"viz_{file_id}.png"
    fig.tight_layout()
    fig.savefig(out, dpi=150)
    plt.close(fig)
    print(out)


plot_timeline("input")

c:\projects\interview\interview-server\ml-server\voice\viz_input.png
